In [1]:
import subprocess
import shlex
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
jar_path = '../SentiStrength.jar'
senti_data_path = '../SentStrength_Data/'

In [3]:
df = pd.read_csv('../../dados/youtube/comentarios1.csv')

In [4]:
def RateSentiment(comments):
    p = subprocess.Popen(shlex.split("java -jar '" + jar_path + "' stdin sentidata '" + senti_data_path + "' trinary"),
                        stdin=subprocess.PIPE,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE)
    
    #Troca todos os espaços, \n, \r, etc, por '+'
    comments = ['+'.join(c) for c in comments.str.split()]
    conc_comments = '\n'.join(comments)
    
    #codifica para mandar
    b = bytes(conc_comments, 'utf-8')
    stdout_byte, stderr_text = p.communicate(b)
    
    #decodifica
    stdout_text = stdout_byte.decode("utf-8")
    
    #troca tab por espaço entre as scores e remove outros espaços
    stdout_text = stdout_text.rstrip().replace("\t"," ").replace("\r","")
    
    return stdout_text.split('\n')

In [5]:
sents_str = RateSentiment(df['text'])

In [6]:
sents_list = [sent.split() for sent in sents_str]
sents_list = [s for s in zip(*sents_list)]

df['sent_pos'] = sents_list[0]
df['sent_neu'] = sents_list[2]
df['sent_neg'] = sents_list[1]

df = df.astype({'sent_pos': int, 'sent_neu': int, 'sent_neg': int})

In [7]:
df.head()

,cid,text,time,author,channel,votes,video_id,parent_id,replies,uploader,uploader_id,video_title,sent_pos,sent_neu,sent_neg
0,UgzBlGJQXIilmNNrc1R4AaABAg,Estou com covid-19 tive sintomas mais no meu p...,2 meses atrás,🔞Canal Marllon YT ⤵️,UCvNuAuBTMykr1V0AvTrjdEg,0,orJy9GCxVHs,NaN,NaN,Dr. Lair Ribeiro Oficial,academialairribeiro,LIVE COVID-19 - PARTE 2,3,-1,-3
1,UgzSflRq03y3bqnVzwJ4AaABAg,"Boa noite Dr. Lair, tudo bem?\nO Senhor aceita...",2 meses atrás,Gladston Cardoso,UCZQ4ckev7OiqZk3FQiWx9iQ,0,orJy9GCxVHs,NaN,NaN,Dr. Lair Ribeiro Oficial,academialairribeiro,LIVE COVID-19 - PARTE 2,2,1,-1
2,UgwNgkRaRBPjOBGsuQh4AaABAg,"Dr Lair sou seu fã, quando puder fale se o aça...",3 meses atrás,Adriano Leal,UCE3KIo9DBVZjnJWL0ie5piw,0,orJy9GCxVHs,NaN,NaN,Dr. Lair Ribeiro Oficial,academialairribeiro,LIVE COVID-19 - PARTE 2,3,-1,-3
3,UgzIrWJMxCaUCHHF4Ed4AaABAg,dr lair eu faco uso do cloreto de magnesio 1 c...,3 meses atrás,anna parreira,UCRevD9obfOXoY5raZQfcycA,0,orJy9GCxVHs,NaN,NaN,Dr. Lair Ribeiro Oficial,academialairribeiro,LIVE COVID-19 - PARTE 2,2,1,-1
4,UgzgLE1od7COmXQxZGN4AaABAg,Oi Dr lair parabéns pelo canal excelente quali...,3 meses atrás,Romero Trindade,UC-h2WEoZnOPFSrHkrg7Ncvg,0,orJy9GCxVHs,NaN,NaN,Dr. Lair Ribeiro Oficial,academialairribeiro,LIVE COVID-19 - PARTE 2,2,-1,-2


In [8]:
df.groupby('uploader').mean()[['sent_pos','sent_neu','sent_neg']]

,sent_pos,sent_neu,sent_neg
uploader,,,
Dr. Alain Dutra,1.591435,-0.077646,-1.652434
Dr. Alvaro Galvão,1.660290,-0.127198,-1.752844
Dr. Felipe Ades MD PhD,1.703125,-0.277344,-2.003906
Dr. Fernando Gomes,1.585018,0.279429,-1.145065
Dr. Lair Ribeiro Oficial,1.779077,-0.024691,-1.684860
Drauzio Varella,1.594018,-0.247698,-1.889809
Julio Pereira - Neurocirurgião,1.684672,-0.221476,-1.917322
Lucy Kerr,1.641905,-0.174984,-1.815619
